## Author: TAN ZHI WEI

In [3]:
import requests  # Add this import statement

# get_tweet_userID

In [8]:
# Import the get_tweet_userID module
import data.get_tweet_userID

def get_user_input():
    return input("Enter the username: ")

def main():
    # Call the main function from get_tweet_userID module
    data.get_tweet_userID.main()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'data.get_tweet_userID'

# fetch_tweet_title


In [7]:
## 22594051 The Star
## 55186601 NST_Online
## 18040230 malaysiakini
## 61083422 theSundaily
## 102098902 Free Malaysia Today 
## 145550026 Herald Malaysia

In [9]:
#------------------------
# Author: Tan Zhi Wei
#------------------------

import requests  # For making HTTP requests
import csv       # For writing data to CSV
import re        # For regular expression to remove URLs

url = "https://twitter241.p.rapidapi.com/user-tweets"
user_ids = ["22594051", "55186601", "18040230", "61083422", "102098902", "145550026"]  # List of user IDs
count_per_user = 50  
headers = {
    "x-rapidapi-key": "6edb283a6cmshee06689689cfd50p13b4c8jsnf64bdb417da6",
    "x-rapidapi-host": "twitter241.p.rapidapi.com"
}
total_tweets_fetched = 0
all_tweets_by_user = {}

# Open a CSV file to write the output
with open('tweets_output.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write the header
    csvwriter.writerow(['User ID', 'Name', 'Followers Count', 'Tweet'])
    
    for user_id in user_ids:
        # We'll potentially make multiple requests with pagination to get more tweets
        cursor = None
        tweets_for_this_user = 0
        full_texts_for_user = []
        
        # Make multiple requests until we reach our desired count or run out of tweets
        while tweets_for_this_user < count_per_user:
            # Add cursor to querystring if we have one
            if cursor:
                querystring = {"user": user_id, "count": "20", "cursor": cursor}
            else:
                querystring = {"user": user_id, "count": "20"}
                
            response = requests.get(url, headers=headers, params=querystring)
            
            # Check if the request was successful
            if response.status_code == 200:
                # Parse the JSON response
                data = response.json()
                
                # Initialize variables for this batch
                batch_tweets = 0
                next_cursor = None
                
                # Extract tweets and look for cursor for pagination
                instructions = data.get("result", {}).get("timeline", {}).get("instructions", [])
                
                for instruction in instructions:
                    if instruction.get("type") == "TimelineAddEntries":
                        entries = instruction.get("entries", [])
                        for entry in entries:
                            # Check if this is a cursor entry
                            if entry.get("entryId", "").startswith("cursor-bottom"):
                                content = entry.get("content", {})
                                if content.get("cursorType") == "Bottom":
                                    next_cursor = content.get("value")
                                continue
                                
                            tweet_result = entry.get("content", {}).get("itemContent", {}).get("tweet_results", {}).get("result", {})
                            full_text = tweet_result.get("legacy", {}).get("full_text", "")
                            user_name = tweet_result.get("core", {}).get("user_results", {}).get("result", {}).get("legacy", {}).get("name", "")
                            followers_count = tweet_result.get("core", {}).get("user_results", {}).get("result", {}).get("legacy", {}).get("followers_count", 0)
                            
                            # Limit followers count to 10 digits
                            if followers_count > 9999999999:
                                followers_count = 9999999999
                            
                            if full_text:
                                # Remove URLs from the full_text and strip whitespace
                                full_text = re.sub(r'http\S+', '', full_text).strip()
                                # Append the full_text to the list
                                full_texts_for_user.append(full_text)
                                # Write to CSV
                                csvwriter.writerow([user_id, user_name, followers_count, full_text])
                                batch_tweets += 1
                                # Break if we've reached our limit
                                if len(full_texts_for_user) >= count_per_user:
                                    break
                
                tweets_for_this_user += batch_tweets
                
                # If we didn't get any tweets in this batch or no next cursor, break
                if batch_tweets == 0 or not next_cursor or len(full_texts_for_user) >= count_per_user:
                    break
                    
                # Set the cursor for the next request
                cursor = next_cursor
                
                # Add a small delay to avoid rate limiting (optional)
                # import time
                # time.sleep(1)
            else:
                print(f"Error for user {user_id}: {response.status_code}")
                print(response.text)
                break
        
        # Store tweets for this user
        all_tweets_by_user[user_id] = full_texts_for_user
        total_tweets_fetched += len(full_texts_for_user)
        
        print(f"Fetched {len(full_texts_for_user)} tweets for user {user_id}")

# Check if we have data for all users
if len(all_tweets_by_user) == len(user_ids):
    print("\nSuccessfully fetched data for all users!")
else:
    print(f"\nFetched data for {len(all_tweets_by_user)} out of {len(user_ids)} users")
print(f"Total tweets fetched: {total_tweets_fetched}")

Fetched 50 tweets for user 22594051
Fetched 50 tweets for user 55186601
Fetched 50 tweets for user 18040230
Fetched 50 tweets for user 61083422
Fetched 50 tweets for user 102098902
Fetched 50 tweets for user 145550026

Successfully fetched data for all users!
Total tweets fetched: 300


# label_data

In [11]:
#------------------------
# Author: Tan Zhi Wei
#------------------------

import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

# Load the CSV file
df = pd.read_csv('tweets_output.csv')

# Initialize the Vader SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to classify sentiment
def classify_sentiment(tweet):
    # Check if the tweet is a string
    if pd.isna(tweet) or not isinstance(tweet, str):
        return 'Unknown'  # Handle NaN or non-string values
    
    sentiment_score = analyzer.polarity_scores(tweet)
    compound_score = sentiment_score['compound']
    
    if compound_score >= 0.05:
        return 'Positive'
    elif compound_score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply sentiment analysis on the 'Tweet' column
df['Sentiment'] = df['Tweet'].apply(classify_sentiment)

# Count the number of tweets with each sentiment
sentiment_counts = df['Sentiment'].value_counts()
print("Sentiment distribution:")
print(sentiment_counts)

# Display the first few rows of the dataframe with sentiment
print("\nSample of tweets with sentiment:")
print(df.head())

# Calculate sentiment distribution by user
print("\nSentiment distribution by user:")
sentiment_by_user = df.groupby(['User ID', 'Sentiment']).size().unstack(fill_value=0)
print(sentiment_by_user)

# Calculate the percentage of positive, negative, and neutral tweets for each user
sentiment_percentage = sentiment_by_user.div(sentiment_by_user.sum(axis=1), axis=0) * 100
print("\nPercentage of sentiment by user:")
print(sentiment_percentage.round(2))

# Save the result to a new CSV
df.to_csv('tweets_output_with_sentiment.csv', index=False)

Sentiment distribution:
Sentiment
Negative    111
Neutral     101
Positive     88
Name: count, dtype: int64

Sample of tweets with sentiment:
    User ID      Name  Followers Count  \
0  22594051  The Star          1903879   
1  22594051  The Star          1903879   
2  22594051  The Star          1903879   
3  22594051  The Star          1903879   
4  22594051  The Star          1903879   

                                               Tweet                Location  \
0   The Umno president said there was no such issue.  Kuala Lumpur, Malaysia   
1  Perikatan Nasional's candidate for the Ayer Ku...  Kuala Lumpur, Malaysia   
2  Police said 77 fatal accident cases were recor...  Kuala Lumpur, Malaysia   
3  It was partially decomposed and may have spent...  Kuala Lumpur, Malaysia   
4  "Retaliatory US-China tariffs are damaging to ...  Kuala Lumpur, Malaysia   

                       Tweet Time  Friends Count Sentiment  
0  Sat Apr 12 05:33:16 +0000 2025            274  Negative  
1 

# kafka_producer


In [14]:
#------------------------
# Author: Tan Zhi Wei
#------------------------

import pandas as pd
from kafka import KafkaProducer
import json

# Define CATEGORY_KEYWORDS dictionary
CATEGORY_KEYWORDS = {
    'PoliticsNewsTopic': [
        'parliament', 'minister', 'government', 'election', 'policy', 'vote', 
        'cabinet', 'PM', 'democracy', 'corruption', 'political', 'politician',
        'law', 'bill', 'constitution', 'amendment', 'opposition', 'campaign',
        'UMNO', 'PAS', 'PKR', 'DAP', 'Bersatu', 'Pakatan', 'Barisan', 'budget'
    ],
    'BusinessNewsTopic': [
        'economy', 'market', 'stock', 'investment', 'company', 'business', 
        'trade', 'finance', 'bank', 'ringgit', 'profit', 'revenue', 'CEO',
        'entrepreneur', 'startup', 'commerce', 'industry', 'economic', 
        'inflation', 'recession', 'growth', 'GST', 'tax', 'BURSA', 'FDI'
    ],
    'SportsNewsTopic': [
        'football', 'badminton', 'hockey', 'athlete', 'tournament', 'championship',
        'league', 'match', 'player', 'coach', 'team', 'sport', 'medal', 'win',
        'game', 'score', 'FIFA', 'Olympic', 'Petronas', 'stadium', 'final',
        'competition', 'record', 'JDT', 'Selangor', 'Perak', 'Malaysia Super League'
    ],
    'EntertainmentNewsTopic': [
        'movie', 'music', 'concert', 'celebrity', 'actor', 'actress', 'film',
        'entertainment', 'drama', 'show', 'artist', 'singer', 'star', 'TV',
        'Netflix', 'performance', 'premiere', 'award', 'festival', 'viral',
        'album', 'song', 'talent', 'meme', 'trending', 'Astro', 'Media Prima'
    ]
}

# Load your CSV data
df = pd.read_csv('tweets_output_with_sentiment.csv')

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Define function to categorize tweets based on keywords
def categorize_tweet(tweet_text):
    # Ensure tweet_text is a string
    if isinstance(tweet_text, str):  # Check if tweet_text is a string
        tweet_text = tweet_text.lower()  # Make the text case-insensitive
        for category, keywords in CATEGORY_KEYWORDS.items():
            if any(keyword in tweet_text for keyword in keywords):  # Check if any keyword matches
                return category
    return 'MalaysiaNewsTopic'  # Default topic if no category is found or invalid tweet

# Iterate and send data to the appropriate Kafka topic
for index, row in df.iterrows():
    tweet_data = {
        'user_id': row['User ID'],
        'name': row['Name'],
        'followers_count': row['Followers Count'],
        'tweet_text': row['Tweet'],
        'sentiment': row['Sentiment']
    }

    # Categorize the tweet into the correct topic
    topic = categorize_tweet(row['Tweet'])

    # Send each row to the appropriate Kafka topic
    producer.send(topic, value=tweet_data)

    # Print confirmation (optional)
    print(f"Sent tweet from {row['Name']} to {topic}")

# Close the producer
producer.flush()
producer.close()
print("All tweets have been sent to Kafka")

Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to PoliticsNewsTopic
Sent tweet from The Star to SportsNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to BusinessNewsTopic
Sent tweet from The Star to BusinessNewsTopic
Sent tweet from The Star to BusinessNewsTopic
Sent tweet from The Star to EntertainmentNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to PoliticsNewsTopic
Sent tweet from The Star to BusinessNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to MalaysiaNewsTopic
Sent tweet from The Star to Pol

# kafka_consumer

In [15]:
#------------------------
# Author: Tan Zhi Wei
#------------------------

import json
from kafka import KafkaConsumer

# Function to consume messages from the selected topic
def consume_messages_from_topic(selected_topic):
    # Initialize Kafka consumer for the selected topic
    consumer = KafkaConsumer(
        selected_topic,  # The topic to verify
        bootstrap_servers=['localhost:9092'],
        auto_offset_reset='earliest',  # Start from the beginning of the topic
        group_id='verification_group',  # A unique consumer group for this verification
        value_deserializer=lambda x: json.loads(x.decode('utf-8')),
        consumer_timeout_ms=10000  # Exit after 10 seconds of no new messages
    )

    print(f"Checking messages in {selected_topic}...")
    message_count = 0

    # Try to consume messages
    for message in consumer:
        message_count += 1
        print(f"Message {message_count}: {message.value}")
        
        # Optional: limit the number of messages to display
        if message_count >= 5:
            print(f"Showing first 5 messages. Topic contains more messages...")
            break

    # Close the consumer
    consumer.close()

    # Check if messages were found
    if message_count > 0:
        print(f"\n✅ Topic verification successful! Found {message_count} messages in {selected_topic}.")
    else:
        print(f"\n❌ No messages found in {selected_topic}. Please check your producer code or Kafka setup.")

# Allow user to choose the topic
available_topics = [
    'PoliticsNewsTopic',
    'BusinessNewsTopic',
    'SportsNewsTopic',
    'EntertainmentNewsTopic',
    'MalaysiaNewsTopic'
]

print("Available Topics to verify:")
for i, topic in enumerate(available_topics, 1):
    print(f"{i}. {topic}")

# Ask the user to select a topic
try:
    topic_choice = int(input("\nEnter the number of the topic you want to verify: "))
    if 1 <= topic_choice <= len(available_topics):
        selected_topic = available_topics[topic_choice - 1]
        consume_messages_from_topic(selected_topic)
    else:
        print("\n❌ Invalid selection. Please choose a valid topic number.")
except ValueError:
    print("\n❌ Invalid input. Please enter a number.")


Available Topics to verify:
1. PoliticsNewsTopic
2. BusinessNewsTopic
3. SportsNewsTopic
4. EntertainmentNewsTopic
5. MalaysiaNewsTopic



Enter the number of the topic you want to verify:  2


Checking messages in BusinessNewsTopic...
Message 1: {'user_id': 22594051, 'name': 'The Star', 'followers_count': 1903879, 'tweet_text': '"Can we have the best of both worlds? Big, sporty, exciting, luxurious cars that are also economical to buy and run?\n\n"Simply put, yes." - Ravindran Kurusamy', 'Location': 'Kuala Lumpur, Malaysia', 'Tweet Time': 'Sat Apr 12 03:40:15 +0000 2025', 'Friends Count': 274, 'sentiment': 'Positive'}
Message 2: {'user_id': 22594051, 'name': 'The Star', 'followers_count': 1903879, 'tweet_text': 'These toys have managed to transcend the traditional toy market, appealing to the emotional and psychological needs of young adults.', 'Location': 'Kuala Lumpur, Malaysia', 'Tweet Time': 'Sat Apr 12 03:30:16 +0000 2025', 'Friends Count': 274, 'sentiment': 'Positive'}
Message 3: {'user_id': 22594051, 'name': 'The Star', 'followers_count': 1903879, 'tweet_text': '"It would be premature to think the dangers posed by the US bond market dysfunction this week have suddenly

# store_data_to_HDFS

In [16]:
#------------------------
# Author: Tan Zhi Wei
#------------------------

import json
import datetime
import uuid
from kafka import KafkaConsumer
from hdfs import InsecureClient

# Initialize Kafka consumer
consumer = KafkaConsumer(
    'MalaysiaNewsTopic',  # Topic name
    bootstrap_servers=['localhost:9092'],
    group_id='tweet_consumer_group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

# Initialize HDFS client
hdfs_client = InsecureClient('http://localhost:9870', user='hduser')

# Specify the HDFS directory
hdfs_directory = '/user/hduser/raw_tweets/'

def write_to_hdfs(data):
    try:
        # Generate unique filename with timestamp and UUID
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        unique_id = str(uuid.uuid4())[:8]  # Use first 8 chars of UUID for brevity
        file_name = f"tweets_raw_{timestamp}_{unique_id}.json"
        
        # Full path to HDFS
        hdfs_path = f"{hdfs_directory}/{file_name}"
        
        # Convert the data to JSON
        data_str = json.dumps(data, ensure_ascii=False)
        
        # Write the data to HDFS
        with hdfs_client.write(hdfs_path, encoding='utf-8') as writer:
            writer.write(data_str)
            
        print(f"Successfully wrote to HDFS: {hdfs_path}")
        return True
    except Exception as e:
        print(f"Error writing to HDFS: {e}")
        return False

# Batch processing option
def write_batch_to_hdfs(batch_data, batch_size=10):
    if not batch_data:
        return True
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    unique_id = str(uuid.uuid4())[:8]
    file_name = f"tweets_batch_{timestamp}_{unique_id}_{batch_size}.json"
    hdfs_path = f"{hdfs_directory}/{file_name}"
    
    data_str = json.dumps(batch_data, ensure_ascii=False)
    
    with hdfs_client.write(hdfs_path, encoding='utf-8') as writer:
        writer.write(data_str)
    
    print(f"Successfully wrote batch of {len(batch_data)} tweets to HDFS")
    return True

# Main processing loop
try:
    # For batch processing
    batch = []
    batch_size = 10
    
    for message in consumer:
        tweet_data = message.value
        
        # Ensure tweet data contains the fields we need
        tweet_data = {
            'user_id': tweet_data['user_id'],
            'name': tweet_data['name'],
            'followers_count': tweet_data['followers_count'],
            'tweet_text': tweet_data['tweet_text'],
            'sentiment': tweet_data.get('sentiment', 'Unknown')  # Include sentiment if available
        }
        
        print(f"Received tweet data: {tweet_data}")
        
        # Write individual tweet data to HDFS
        write_to_hdfs(tweet_data)
        
        # Batch processing logic
        batch.append(tweet_data)
        if len(batch) >= batch_size:
            write_batch_to_hdfs(batch, batch_size)
            batch = []  # Reset batch after writing
    
except KeyboardInterrupt:
    print("Stopping consumer...")
finally:
    consumer.close()
    print("Consumer closed.")

Stopping consumer...


KeyboardInterrupt: 

In [ ]:
# dont RUN !

In [ ]:
# check data exits or not
# hadoop fs -ls /user/hduser/raw_tweets/  
# hadoop fs -cat /user/hduser/raw_tweets//tweets_raw_20250314_171804_7933c577.json
